# AiiDAlab QauntumESPRESSO App

<font color="red"><b>Caution!</b></font> Deleting this job will also remove <b>all associated nodes</b>, including any related calculations and their respective results. This action is <b>irreversible</b>.


In [ ]:
import urllib.parse as urlparse

import ipywidgets as widgets
from aiida import load_profile
from aiida.orm import load_node
from aiida.tools import delete_nodes
from IPython.display import Markdown, display

# Load AiiDA profile
load_profile()

# Parse the primary key from the Jupyter notebook URL
url = urlparse.urlsplit(jupyter_notebook_url) # noqa F821
query = urlparse.parse_qs(url.query)
pk = int(query['pk'][0])

# Function to display node details


def display_node_details(pk):
    try:
        node = load_node(pk)
        print(f"Node ID: {node.pk}")
        print(f"Node Type: {node.process_label}")
        print(f"Label: {node.label}")
        print(f"Description: {node.description}")
        print(f"Creation Time: {node.ctime}")
    except Exception as e:
        print(f"Error loading node: {str(e)}")
        return False
    return True

# Function to delete node


def delete_node(pk, dry_run=True):
    if dry_run:
        _, was_deleted = delete_nodes([pk], dry_run=True)
        if was_deleted:
            print(f'Dry run: Node {pk} can be deleted.')
        return
    
    _, was_deleted = delete_nodes([pk], dry_run=False)
    if was_deleted:
        print(f'Node {pk} deleted successfully.')

# User interaction for node deletion


def confirm_deletion(b):
    if delete_confirmation.value.lower() in ['y', 'yes']:
        delete_node(pk, dry_run=False)
    else:
        print('Deletion aborted.')


# Display node details
if display_node_details(pk):
    # Ask for confirmation
    nodes, _ = delete_nodes([pk], dry_run=False)
    display(Markdown(f'**YOU ARE ABOUT TO DELETE `{len(nodes)}` NODES! THIS CANNOT BE UNDONE!**'))
    delete_confirmation = widgets.Text(
        value='',
        placeholder='Type "yes" to confirm',
        description='Confirm:',
        disabled=False
    )
    confirm_button = widgets.Button(description="Delete Node")
    confirm_button.on_click(confirm_deletion)
    display(delete_confirmation, confirm_button)
else:
    print("No valid node found for deletion.")